1

In [8]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [9]:
%%bigquery --project numeric-mote-259221
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


In [0]:
%%bigquery --project numeric-mote-259221 df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [11]:
%%bigquery --project numeric-mote-259221

CREATE OR REPLACE MODEL `bqml_tutorial.sample_model` 
OPTIONS(model_type='logistic_reg') AS
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170631'
LIMIT 100000;

""


In [12]:
%%bigquery --project numeric-mote-259221

#standardSQL
SELECT
  *
FROM
  ml.EVALUATE(MODEL `bqml_tutorial.sample_model`, (
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'));

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.468085,0.081937,0.985397,0.139461,0.04623,0.979032


In [13]:
%%bigquery --project numeric-mote-259221

#standardSQL
SELECT
  country,
  SUM(predicted_label) as total_predicted_purchases
FROM
  ml.PREDICT(MODEL `bqml_tutorial.sample_model`, (
SELECT
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(totals.pageviews, 0) AS pageviews,
  IFNULL(geoNetwork.country, "") AS country
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
GROUP BY country
ORDER BY total_predicted_purchases DESC
LIMIT 10;

,country,total_predicted_purchases
0,United States,167
1,Taiwan,5
2,Canada,3
3,India,2
4,Indonesia,1
5,Japan,1
6,Brazil,1
7,Germany,1
8,Venezuela,1
9,Turkey,1


In [14]:
%%bigquery --project numeric-mote-259221

#standardSQL
SELECT
  fullVisitorId,
  SUM(predicted_label) as total_predicted_purchases
FROM
  ml.PREDICT(MODEL `bqml_tutorial.sample_model`, (
SELECT
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(totals.pageviews, 0) AS pageviews,
  IFNULL(geoNetwork.country, "") AS country,
  fullVisitorId
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
GROUP BY fullVisitorId
ORDER BY total_predicted_purchases DESC
LIMIT 10;

,fullVisitorId,total_predicted_purchases
0,9417857471295131045,3
1,112288330928895942,2
2,2969418676126258798,2
3,7420300501523012460,2
4,2105122376016897629,2
5,0376394056092189113,2
6,1280993661204347450,2
7,8388931032955052746,2
8,057693500927581077,2
9,806992249032686650,2


2

In [15]:
%%bigquery --project numeric-mote-259221
SELECT
  *
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`
LIMIT
  100;

,age,workclass,functional_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,Private,297847,9th,5,Married-civ-spouse,Other-service,Wife,Black,Female,3411,0,34,United-States,<=50K
1,72,Private,74141,9th,5,Married-civ-spouse,Exec-managerial,Wife,Asian-Pac-Islander,Female,0,0,48,United-States,>50K
2,45,Private,178215,9th,5,Married-civ-spouse,Machine-op-inspct,Wife,White,Female,0,0,40,United-States,>50K
3,31,Private,86958,9th,5,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
4,55,Private,176012,9th,5,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,23,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,30,Local-gov,48542,12th,8,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,40,United-States,<=50K
96,53,?,155233,12th,8,Married-civ-spouse,?,Wife,White,Female,0,0,40,Italy,<=50K
97,52,?,89951,12th,8,Married-civ-spouse,?,Wife,Black,Female,0,0,40,United-States,>50K
98,45,Private,81400,1st-4th,2,Married-civ-spouse,Other-service,Wife,White,Female,0,0,25,El-Salvador,<=50K


In [16]:
%%bigquery --project numeric-mote-259221

CREATE OR REPLACE VIEW
  `census.input_view` AS
SELECT
  age,
  workclass,
  native_country,
  marital_status,
  education_num,
  occupation,
  race,
  hours_per_week,
  income_bracket,
  CASE
    WHEN MOD(functional_weight, 10) < 8 THEN 'training'
    WHEN MOD(functional_weight, 10) = 8 THEN 'evaluation'
    WHEN MOD(functional_weight, 10) = 9 THEN 'prediction'
  END AS dataframe
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`

""


In [17]:
%%bigquery --project numeric-mote-259221

CREATE OR REPLACE MODEL
  `census.census_model`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    data_split_method='NO_SPLIT',
    input_label_cols=['income_bracket'],
    max_iterations=15) AS
SELECT
  *
FROM
  `census.input_view`
WHERE
  dataframe = 'training'

""


In [18]:
%%bigquery --project numeric-mote-259221

SELECT
  *
FROM
  ML.EVALUATE (MODEL `census.census_model`,
    (
    SELECT
      *
    FROM
      `census.input_view`
    WHERE
      dataframe = 'evaluation'
    )
  )

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.553977,0.794837,0.807192,0.652902,0.412177,0.882952


In [19]:
%%bigquery --project numeric-mote-259221

SELECT
  *
FROM
  ML.PREDICT (MODEL `census.census_model`,
    (
    SELECT
      *
    FROM
      `census.input_view`
    WHERE
      dataframe = 'prediction'
     )
  )

,predicted_income_bracket,predicted_income_bracket_probs,age,workclass,native_country,marital_status,education_num,occupation,race,hours_per_week,income_bracket,dataframe
0,<=50K,"[{'label': ' >50K', 'prob': 0.1891482573116060...",52,Local-gov,United-States,Married-civ-spouse,5,Other-service,White,40,<=50K,prediction
1,<=50K,"[{'label': ' >50K', 'prob': 0.0917142831148600...",34,?,United-States,Married-civ-spouse,7,?,White,8,<=50K,prediction
2,<=50K,"[{'label': ' >50K', 'prob': 0.2978379336023622...",47,Private,United-States,Married-civ-spouse,7,Machine-op-inspct,Black,40,<=50K,prediction
3,<=50K,"[{'label': ' >50K', 'prob': 0.1744424535409649...",43,Private,United-States,Married-civ-spouse,7,Handlers-cleaners,White,25,<=50K,prediction
4,<=50K,"[{'label': ' >50K', 'prob': 0.3223656785315964...",54,Private,United-States,Married-civ-spouse,7,Other-service,White,43,<=50K,prediction
...,...,...,...,...,...,...,...,...,...,...,...,...
3137,<=50K,"[{'label': ' >50K', 'prob': 0.0359979067711647...",23,Private,United-States,Never-married,10,Other-service,White,40,<=50K,prediction
3138,<=50K,"[{'label': ' >50K', 'prob': 0.0521848424868671...",35,Private,United-States,Never-married,10,Machine-op-inspct,Amer-Indian-Eskimo,40,<=50K,prediction
3139,<=50K,"[{'label': ' >50K', 'prob': 0.0280517115841698...",19,?,United-States,Never-married,10,?,White,30,<=50K,prediction
3140,<=50K,"[{'label': ' >50K', 'prob': 0.0276128468409307...",26,Private,?,Never-married,10,Handlers-cleaners,Black,40,<=50K,prediction


3

In [20]:
%%bigquery --project numeric-mote-259221

WITH
  hs AS (
  SELECT
    h.start_station_name AS station_name,
    h.duration,
    ST_DISTANCE(ST_GEOGPOINT(s.longitude,
        s.latitude),
      ST_GEOGPOINT(-0.1,
        51.5))/1000 AS distance_from_city_center
  FROM
    `bigquery-public-data.london_bicycles.cycle_hire` AS h
  JOIN
    `bigquery-public-data.london_bicycles.cycle_stations` AS s
  ON
    h.start_station_id = s.id
  WHERE
    h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
  stationstats AS (
  SELECT
    station_name,
    AVG(duration) AS duration,
    COUNT(duration) AS num_trips,
    MAX(distance_from_city_center) AS distance_from_city_center
  FROM
    hs
  GROUP BY
    station_name )
SELECT
  *
FROM
  stationstats
ORDER BY
  distance_from_city_center ASC

,station_name,duration,num_trips,distance_from_city_center
0,"Borough Road, Elephant & Castle",1349.318091,7523,0.126240
1,"Webber Street , Southwark",823.879056,8136,0.164021
2,"Great Suffolk Street, The Borough",844.355487,10442,0.193667
3,"LSBU (Borough Road), Elephant & Castle",1297.835314,7068,0.257903
4,"Harper Road, Borough",822.686747,1660,0.306306
...,...,...,...,...
789,"Bridge Avenue, Hammersmith",2269.098519,9318,8.981545
790,"Montserrat Road , Putney",2648.183079,4326,8.986740
791,"Disraeli Road, Putney",1777.219098,8357,9.105643
792,"Putney Rail Station, Putney",3715.898645,3394,9.175861


In [21]:
%%bigquery --project numeric-mote-259221

CREATE OR REPLACE MODEL
  bqml_tutorial.london_station_clusters OPTIONS(model_type='kmeans',
    num_clusters=4) AS
WITH
  hs AS (
  SELECT
    h.start_station_name AS station_name,
  IF
    (EXTRACT(DAYOFWEEK
      FROM
        h.start_date) = 1
      OR EXTRACT(DAYOFWEEK
      FROM
        h.start_date) = 7,
      "weekend",
      "weekday") AS isweekday,
    h.duration,
    ST_DISTANCE(ST_GEOGPOINT(s.longitude,
        s.latitude),
      ST_GEOGPOINT(-0.1,
        51.5))/1000 AS distance_from_city_center
  FROM
    `bigquery-public-data.london_bicycles.cycle_hire` AS h
  JOIN
    `bigquery-public-data.london_bicycles.cycle_stations` AS s
  ON
    h.start_station_id = s.id
  WHERE
    h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
  stationstats AS (
  SELECT
    station_name,
    isweekday,
    AVG(duration) AS duration,
    COUNT(duration) AS num_trips,
    MAX(distance_from_city_center) AS distance_from_city_center
  FROM
    hs
  GROUP BY
    station_name, isweekday)
SELECT
  * EXCEPT(station_name, isweekday)
FROM
  stationstats

""


In [22]:
%%bigquery --project numeric-mote-259221

WITH
  hs AS (
  SELECT
    h.start_station_name AS station_name,
    h.duration,
    ST_DISTANCE(ST_GEOGPOINT(s.longitude,
        s.latitude),
      ST_GEOGPOINT(-0.1,
        51.5))/1000 AS distance_from_city_center
  FROM
    `bigquery-public-data.london_bicycles.cycle_hire` AS h
  JOIN
    `bigquery-public-data.london_bicycles.cycle_stations` AS s
  ON
    h.start_station_id = s.id
  WHERE
    h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
  stationstats AS (
  SELECT
    station_name,
    AVG(duration) AS duration,
    COUNT(duration) AS num_trips,
    MAX(distance_from_city_center) AS distance_from_city_center
  FROM
    hs
  GROUP BY
    station_name )
SELECT
  * EXCEPT(nearest_centroids_distance)
FROM
  ML.PREDICT( MODEL bqml_tutorial.london_station_clusters,
    (
    SELECT
      *
    FROM
      stationstats
    WHERE
      REGEXP_CONTAINS(station_name, 'Kennington')))

,CENTROID_ID,station_name,duration,num_trips,distance_from_city_center
0,2,"Kennington Road Post Office, Oval",2096.259542,7074,1.846033
1,2,"Cleaver Street, Kennington",1231.158015,5259,1.496792
2,2,"Doddington Grove, Kennington",1532.156098,9225,1.468141
3,3,"Kennington Lane Rail Bridge, Vauxhall",1144.593108,25277,2.175033
4,2,"Kennington Oval, Oval",1325.593913,12485,2.083134
5,3,"Kennington Cross, Kennington",964.901218,19538,1.462588
6,2,"Kennington Road , Vauxhall",1346.311431,11110,0.891565
7,2,"Kennington Station, Kennington",2209.022758,6723,1.298668
8,2,"Cotton Garden Estate, Kennington",1439.843773,6913,1.117034


4

In [0]:
%%bigquery --project numeric-mote-259221

CREATE OR REPLACE MODEL bqml_tutorial.ncaa_model OPTIONS(
  model_type='linear_reg', max_iteration=50) AS
SELECT
  * EXCEPT(
    game_id, season, scheduled_date,
    total_three_points_made,
    total_three_points_att),
  total_three_points_att as label
FROM
  `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE
  # remove the game to predict
  game_id != 'f1063e80-23c7-486b-9a5e-faa52beb2d83'

In [0]:
%%bigquery --project numeric-mote-259221

SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_tutorial.ncaa_model`)

In [0]:
%%bigquery --project numeric-mote-259221

WITH eval_table AS (
SELECT
  *,
  total_three_points_att AS label
FROM
  `bqml_tutorial.wide_games` )
SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_tutorial.ncaa_model`,
    TABLE eval_table)

In [0]:
%%bigquery --project numeric-mote-259221

  WITH game_to_predict AS (
  SELECT
    *
  FROM
    `bqml_tutorial.wide_games`
  WHERE
    game_id='f1063e80-23c7-486b-9a5e-faa52beb2d83' )
SELECT
  truth.game_id AS game_id,
  total_three_points_att,
  predicted_total_three_points_att
FROM (
  SELECT
    game_id,
    predicted_label AS predicted_total_three_points_att
  FROM
    ML.PREDICT(MODEL `bqml_tutorial.ncaa_model`,
      table game_to_predict) ) AS predict
JOIN (
  SELECT
    game_id,
    total_three_points_att AS total_three_points_att
  FROM
    game_to_predict) AS truth
ON
  predict.game_id = truth.game_id

5

In [26]:
%%bigquery --project numeric-mote-259221

CREATE MODEL `bqml_tutorial.natality_model`
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['weight_pounds']) AS
SELECT
  weight_pounds,
  is_male,
  gestation_weeks,
  mother_age,
  CAST(mother_race AS string) AS mother_race
FROM
  `bigquery-public-data.samples.natality`
WHERE
  weight_pounds IS NOT NULL
  AND RAND() < 0.001

""


In [27]:
%%bigquery --project numeric-mote-259221

SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_tutorial.natality_model`,
    (
    SELECT
      weight_pounds,
      is_male,
      gestation_weeks,
      mother_age,
      CAST(mother_race AS STRING) AS mother_race
    FROM
      `bigquery-public-data.samples.natality`
    WHERE
      weight_pounds IS NOT NULL))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.956419,1.675675,0.034258,0.738557,0.046483,0.046484


In [28]:
%%bigquery --project numeric-mote-259221

SELECT
  predicted_weight_pounds
FROM
  ML.PREDICT(MODEL `bqml_tutorial.natality_model`,
    (
    SELECT
      is_male,
      gestation_weeks,
      mother_age,
      CAST(mother_race AS STRING) AS mother_race
    FROM
      `bigquery-public-data.samples.natality`
    WHERE
      state = "WY"))

,predicted_weight_pounds
0,7.745286
1,7.655229
2,7.465117
3,7.703034
4,7.417312
...,...
218730,7.184083
218731,7.184083
218732,7.184083
218733,7.184083


AUTO ML TABLES

1

In [70]:
PROJECT_ID = "numeric-mote-259221"
COMPUTE_REGION = "us-central1"
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [71]:
import sys
if 'google.colab' in sys.modules:    
  from google.colab import files
  keyfile_upload = files.upload()
  keyfile = list(keyfile_upload.keys())[0]
  %env GOOGLE_APPLICATION_CREDENTIALS $keyfile
else:
  %env GOOGLE_APPLICATION_CREDENTIALS "/Users/nsy/Documents/Grad2/5671/HW11/service_account.json"

Saving service_account.json to service_account (2).json
env: GOOGLE_APPLICATION_CREDENTIALS=service_account.json


In [0]:
! pip3 install --upgrade --quiet google-cloud-automl google-cloud-bigquery

In [72]:
%pip install google-cloud-automl

In [0]:
from google.cloud import automl_v1beta1 as automl

import google.cloud.automl_v1beta1.proto.data_types_pb2 as data_types
import matplotlib.pyplot as plt

In [0]:
client = automl.TablesClient(project=PROJECT_ID, region=COMPUTE_REGION)

In [75]:
list_models = client.list_models()
models = { model.display_name: model.name for model in list_models }
models

{}

In [76]:
dataset_display_name = 'census' 
dataset = client.create_dataset(dataset_display_name)
dataset

name: "projects/269327017718/locations/us-central1/datasets/TBL797079959439933440"
display_name: "census"
create_time {
  seconds: 1573957198
  nanos: 286438000
}
etag: "AB3BwFr45DO_uRX_fXA9_FdySGyt3p9Us9KaJFcoILE3gFJoy5FihjNJ038A4z5MMTVV"
tables_dataset_metadata {
  stats_update_time {
  }
}

In [77]:
GCS_STORAGE_BUCKET = 'gs://{}-codelab-data-storage'.format(PROJECT_ID)
GCS_DATASET_URI = '{}/census_income.csv'.format(GCS_STORAGE_BUCKET)
! gsutil ls $GCS_STORAGE_BUCKET || gsutil mb -l $COMPUTE_REGION $GCS_STORAGE_BUCKET
! gsutil cp gs://cloud-ml-data-tables/notebooks/census_income.csv $GCS_DATASET_URI

gs://numeric-mote-259221-codelab-data-storage/census_income.csv
Copying gs://cloud-ml-data-tables/notebooks/census_income.csv [Content-Type=application/octet-stream]...
/ [1 files][  3.8 MiB/  3.8 MiB]                                                
Operation completed over 1 objects/3.8 MiB.                                      


In [0]:
import_data_operation = client.import_data(
    dataset=dataset,
    gcs_input_uris = gcs_input_uris
)
print('Dataset import operation: {}'.format(import_data_operation))

In [0]:
list_table_specs_response = client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]

# List column specs
list_column_specs_response = client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print Features and data_type:

features = [(key, data_types.TypeCode.Name(value.data_type.type_code)) for key, value in column_specs.items()]
print('Feature list:\n')
for feature in features:
    print(feature[0],':', feature[1])

In [0]:
type_counts = {}
for column_spec in column_specs.values():
  type_name = data_types.TypeCode.Name(column_spec.data_type.type_code)
  type_counts[type_name] = type_counts.get(type_name, 0) + 1
    
plt.pie(x=type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In [0]:
update_column_response = client.update_column_spec(
    dataset=dataset,
    column_spec_display_name='income',
    type_code='CATEGORY',
    nullable=False,
)
update_column_response

In [0]:
update_dataset_response = client.set_target_column(
    dataset=dataset,
    column_spec_display_name='income',
)
update_dataset_response

In [0]:
model_display_name = 'census_income_model'

create_model_response = client.create_model(
    model_display_name,
    dataset=dataset,
    train_budget_milli_node_hours=1000,
)
print('Create model operation: {}'.format(create_model_response.operation))
# Wait until model training is done.
model = create_model_response.result()
model

In [0]:
client.deploy_model(model=model).result()

In [0]:
model = client.get_model(model_name=model.name)
model

In [0]:
from ipywidgets import interact
import ipywidgets as widgets

workclass_ids = ['Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov', 'State-gov', 'Without-pay', 'Never-worked']
education_ids = ['Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', '9th', '7th-8th', '12th', 'Masters', '1st-4th', '10th', 'Doctorate', '5th-6th', 'Preschool']
marital_status_ids = ['Married-civ-spouse', 'Divorced', 'Never-married', 'Separated', 'Widowed', 'Married-spouse-absent', 'Married-AF-spouse']
occupation_ids = ['Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Armed-Forces']
relationship_ids = ['Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative', 'Unmarried']
race_ids = ['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black']
sex_ids = ['Female', 'Male']
native_country_ids = ['United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada', 'Germany', 'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece', 'South', 'China', 'Cuba', 'Iran', 'Honduras', 'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam', 'Mexico', 'Portugal', 'Ireland', 'France', 'Dominican-Republic', 'Laos', 'Ecuador', 'Taiwan', 'Haiti', 'Columbia', 'Hungary', 'Guatemala', 'Nicaragua', 'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago', 'Peru', 'Hong', 'Holand-Netherlands']

workclass = widgets.Dropdown(
    options=workclass_ids, 
    value=workclass_ids[0],
    description='workclass:'
)

education = widgets.Dropdown(
    options=education_ids, 
    value=education_ids[0],
    description='education:', 
    width='500px'
)
 
marital_status = widgets.Dropdown(
    options=marital_status_ids, 
    value=marital_status_ids[0],
    description='marital status:', 
    width='500px'
)

occupation = widgets.Dropdown(
    options=occupation_ids, 
    value=occupation_ids[0],
    description='occupation:', 
    width='500px'
)

relationship = widgets.Dropdown(
    options=relationship_ids, 
    value=relationship_ids[0],
    description='relationship:', 
    width='500px'
)

race = widgets.Dropdown(
    options=race_ids, 
    value=race_ids[0],                           
    description='race:', 
    width='500px'
)

sex = widgets.Dropdown(
    options=sex_ids, 
    value=sex_ids[0],
    description='sex:', 
    width='500px'
)

native_country = widgets.Dropdown(
    options=native_country_ids, 
    value=native_country_ids[0],
    description='native_country:', 
    width='500px'
)

display(workclass)
display(education)
display(marital_status)
display(occupation)
display(relationship)
display(race)
display(sex)
display(native_country)

In [0]:
age = 34 #@param {type:'slider', min:1, max:100, step:1}
capital_gain = 40000 #@param {type:'slider', min:0, max:100000, step:10000}
capital_loss = 3.8 #@param {type:'slider', min:0, max:4000, step:0.1}
fnlwgt = 150000 #@param {type:'slider', min:0, max:1000000, step:50000}
education_num = 9 #@param {type:'slider', min:1, max:16, step:1}
hours_per_week = 40 #@param {type:'slider', min:1, max:100, step:1}

In [0]:
inputs = {
    'age': age,
    'workclass': workclass.value,
    'fnlwgt': fnlwgt,
    'education': education.value,
    'education_num': education_num,
    'marital_status': marital_status.value,
    'occupation': occupation.value,
    'relationship': relationship.value,
    'race': race.value,
    'sex': sex.value,
    'capital_gain': capital_gain,
    'capital_loss': capital_loss,
    'hours_per_week': hours_per_week,
    'native_country': native_country.value,
}

prediction_result = client.predict(model=model, inputs=inputs)
prediction_result

In [0]:
predictions = [(prediction.tables.score, prediction.tables.value.string_value) for prediction in prediction_result.payload]
predictions = sorted(predictions, key=lambda tup: (tup[0],tup[1]), reverse=True)
print('Prediction is: ', predictions[0])

In [0]:
undeploy_model_response = client.undeploy_model(model=model)

In [0]:
GCS_BATCH_PREDICT_URI = '{}/census_income_batch_prediction_input.csv'.format(GCS_STORAGE_BUCKET)
GCS_BATCH_PREDICT_OUTPUT = '{}/census_income_predictions/'.format(GCS_STORAGE_BUCKET)
! gsutil cp gs://cloud-ml-data-tables/notebooks/census_income_batch_prediction_input.csv $GCS_BATCH_PREDICT_URI

In [0]:
batch_predict_response = client.batch_predict(
    model=model, 
    gcs_input_uris=GCS_BATCH_PREDICT_URI,
    gcs_output_uri_prefix=GCS_BATCH_PREDICT_OUTPUT,
)
print('Batch prediction operation: {}'.format(batch_predict_response.operation))
# Wait until batch prediction is done.
batch_predict_result = batch_predict_response.result()
batch_predict_response.metadata

2

In [100]:
client = automl.TablesClient(project=PROJECT_ID, region=COMPUTE_REGION)
dataset_display_name = 'colab_trial1' #@param {type: 'string'}

dataset = client.create_dataset(dataset_display_name)
dataset

name: "projects/269327017718/locations/us-central1/datasets/TBL5183585996498796544"
display_name: "colab_trial1"
create_time {
  seconds: 1573960553
  nanos: 568482000
}
etag: "AB3BwFrunFvPKM9hPPKPu220gGr2EkmPTyuRBvlPNxXMJXgfrJcFJQnUQrP1nCTcN1wm"
tables_dataset_metadata {
  stats_update_time {
  }
}

In [0]:
bucket_name = 'swjsock' #@param {type: 'string'}

In [102]:
!sudo pip install google-cloud-bigquery google-cloud-storage pandas pandas-gbq gcsfs oauth2client

import datetime
import pandas as pd

import gcsfs
from google.cloud import bigquery
from google.cloud import storage

client_bq = bigquery.Client(location='US', project=PROJECT_ID)

In [0]:
def balanceTable(table):
    #class count
  count_class_false, count_class_true = table.totalTransactionRevenue.value_counts()

    #divide by class
  table_class_false = table[table["totalTransactionRevenue"] == False]
  table_class_true = table[table["totalTransactionRevenue"] == True]

    #random over-sampling
  table_class_true_over = table_class_true.sample(count_class_false, replace = True)
  table_test_over = pd.concat([table_class_false, table_class_true_over])
  return table_test_over


def partitionTable(table, dt=20170500):
  #the automl tables model could be training on future data and implicitly learning about past data in the testing
  #dataset, this would cause data leakage. To prevent this, we are training only with the first 9 months of data (table1)
  #and doing validation with the last three months of data (table2).
  table1 = table[table["date"] <= dt]
  table2 = table[table["date"] > dt]
  return table1, table2

def N_updatePrevCount(table, new_column, old_column):
  table = table.fillna(0)
  table[new_column] = 1
  table.sort_values(by=['fullVisitorId','date'])
  table[new_column] = table.groupby(['fullVisitorId'])[old_column].apply(lambda x: x.cumsum())
  table.drop([old_column], axis = 1, inplace = True)
  return table


def N_updateDate(table):
  table['weekday'] = 1
  table['date'] = pd.to_datetime(table['date'].astype(str), format = '%Y%m%d')
  table['weekday'] = table['date'].dt.dayofweek
  return table


def change_transaction_values(table):
  table['totalTransactionRevenue'] = table['totalTransactionRevenue'].fillna(0)
  table['totalTransactionRevenue'] = table['totalTransactionRevenue'].apply(lambda x: x!=0)
  return table

def saveTable(table, csv_file_name, bucket_name):
  table.to_csv(csv_file_name, index = False)
  storage_client = storage.Client()
  bucket = storage_client.get_bucket(bucket_name)
  blob = bucket.blob(csv_file_name)
  blob.upload_from_filename(filename = csv_file_name)

In [0]:
unnested_file_name = "FULL_unnested.csv"

In [0]:
table = pd.read_csv("gs://"+bucket_name+"/"+unnested_file_name, low_memory=False)

column_names = ['device', 'geoNetwork','totals', 'trafficSource']

for name in column_names:
  print(name)
  table[name] = table[name].apply(lambda i: dict(eval(i)))
  temp = table[name].apply(pd.Series)
  table = pd.concat([table, temp], axis=1).drop(name, axis=1)

#need to drop a column
table.drop(['adwordsClickInfo'], axis = 1, inplace = True)
saveTable(table, unnested_file_name, bucket_name)

In [0]:
dataset_gcs_input_uris = ['gs://{}/training_unnested_balanced_FULL.csv'.format(bucket_name),] #@param
import_data_operation = client.import_data(
    dataset=dataset,
    gcs_input_uris=dataset_gcs_input_uris
)
print('Dataset import operation: {}'.format(import_data_operation))

# Synchronous check of operation status. Wait until import is done.
import_data_operation.result()
dataset = client.get_dataset(dataset_name=dataset.name)
dataset

In [0]:
list_table_specs_response = client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]

# List column specs
list_column_specs_response = client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print Features and data_type:

features = [(key, data_types.TypeCode.Name(value.data_type.type_code)) for key, value in column_specs.items()]
print('Feature list:\n')
for feature in features:
    print(feature[0],':', feature[1])
    
# Table schema pie chart.

type_counts = {}
for column_spec in column_specs.values():
  type_name = data_types.TypeCode.Name(column_spec.data_type.type_code)
  type_counts[type_name] = type_counts.get(type_name, 0) + 1
    
plt.pie(x=type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In [0]:
update_column_response = client.update_column_spec(
    dataset=dataset,
    column_spec_display_name='totalTransactionRevenue',
    nullable=False,
)
update_column_response

In [0]:
update_dataset_response = client.set_target_column(
    dataset=dataset,
    column_spec_display_name='totalTransactionRevenue',
)
update_dataset_response

In [0]:
model_display_name = 'trial_1' #@param {type:'string'}

create_model_response = client.create_model(
    model_display_name,
    dataset=dataset,
    train_budget_milli_node_hours=1000,
)
print('Create model operation: {}'.format(create_model_response.operation))
# Wait until model training is done.
model = create_model_response.result()
model

In [0]:
batch_predict_gcs_input_uris = ['gs://cloud-ml-data-tables/notebooks/validation_unnested_FULL.csv',] #@param
batch_predict_gcs_output_uri_prefix = 'gs://{}'.format(bucket_name) #@param {type:'string'}
batch_predict_response = client.batch_predict(
    model=model, 
    gcs_input_uris=batch_predict_gcs_input_uris,
    gcs_output_uri_prefix=batch_predict_gcs_output_uri_prefix,
)
print('Batch prediction operation: {}'.format(batch_predict_response.operation))
# Wait until batch prediction is done.
batch_predict_result = batch_predict_response.result()
batch_predict_response.metadata

In [0]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

def invert(x):
  return 1-x

def switch_label(x):
  return(not x)
batch_predict_results_location = 'gs://<your-test-bucket>/' #@param {type:'string'}

table = pd.read_csv(batch_predict_results_location +'/<resultid>/tables_1.csv')
y = table["totalTransactionRevenue"]
scores = table["totalTransactionRevenue_1.0_score"]
scores_invert = table['totalTransactionRevenue_0.0_score']

#code for ROC curve, for true values
fpr, tpr, thresholds = metrics.roc_curve(y, scores)
roc_auc = metrics.auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for True')
plt.legend(loc="lower right")
plt.show()


#code for ROC curve, for false values
plt.figure()
lw = 2
label_invert = y.apply(switch_label)
fpr, tpr, thresholds = metrics.roc_curve(label_invert, scores_invert)
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for False')
plt.legend(loc="lower right")
plt.show()


#code for PR curve, for true values

precision, recall, thresholds = metrics.precision_recall_curve(y, scores)


plt.figure()
lw = 2
plt.plot( recall, precision, color='darkorange',
         lw=lw, label='Precision recall curve for True')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve for True')
plt.legend(loc="lower right")
plt.show()

#code for PR curve, for false values

precision, recall, thresholds = metrics.precision_recall_curve(label_invert, scores_invert)
print(precision.shape)
print(recall.shape)

plt.figure()
lw = 2
plt.plot( recall, precision, color='darkorange',
          label='Precision recall curve for False')
plt.xlim([0.0, 1.1])
plt.ylim([0.0, 1.1])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve for False')
plt.legend(loc="lower right")
plt.show()

3

In [0]:
project_id = 'numeric-mote-259221' #@param {type:'string'}
location = 'us-central1' #@param {type:'string'}

In [108]:
from google.oauth2 import service_account
from google.colab import files
from google.cloud import automl_v1beta1

# Upload service account key
keyfile_upload = files.upload()
keyfile_name = list(keyfile_upload.keys())[0]
# Authenticate and create an AutoML client.
credentials = service_account.Credentials.from_service_account_file(keyfile_name)
client = automl.TablesClient(project=project_id, region=location, credentials=credentials)

Saving service_account.json to service_account (3).json


In [109]:
dataset_display_name = 'energy_forcasting_solution' 
dataset = client.create_dataset(dataset_display_name)
print(dataset.name) # unique to this dataset
dataset

projects/269327017718/locations/us-central1/datasets/TBL4555439401596878848


name: "projects/269327017718/locations/us-central1/datasets/TBL4555439401596878848"
display_name: "energy_forcasting_solution"
create_time {
  seconds: 1573961007
  nanos: 171636000
}
etag: "AB3BwFqae4FrmqUfTgbQ4ATSv6TGAfW3SgFlujhDTN9EAebyN3gFHAdAARjfmQBhkpk="
tables_dataset_metadata {
  stats_update_time {
  }
}

In [0]:
dataset = client.get_dataset(dataset_display_name=dataset_display_name)

In [0]:
dataset_bq_input_uri = 'bq://energy-forecasting.Energy.automldata' #@param {type: 'string'}

import_data_operation = client.import_data(
    dataset=dataset,
    bigquery_input_uri=dataset_bq_input_uri
)

print('Dataset import operation: {}'.format(import_data_response.operation))

# Wait until import is done.
import_data_result = import_data_response.result()
import_data_result

In [0]:
list_table_specs_response = client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]

# List column specs
list_column_specs_response = client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print Features and data_type:

features = [(key, data_types.TypeCode.Name(value.data_type.type_code)) for key, value in column_specs.items()]
print('Feature list:\n')
for feature in features:
    print(feature[0],':', feature[1])

In [0]:
column_to_category = 'hour' #@param {type: 'string'}

update_column_response = client.update_column_spec(
    dataset=dataset,
    column_spec_display_name=column_to_category,
    type_code='CATEGORY'
)

update_column_response.display_name, update_column_response.data_type

In [0]:
target_column_name = 'price' #@param {type: 'string'}
split_column_name = 'split' #@param {type: 'string'}

client.set_target_column(
    dataset=dataset,
    column_spec_display_name=target_column_name,
)

client.set_test_train_column(
    dataset=dataset,
    column_spec_display_name=split_column_name,
)

In [0]:
model_display_name = 'energy_model' #@param {type:'string'}
model_train_hours = 12 #@param {type:'integer'}
model_optimization_objective = 'MINIMIZE_MAE' #@param {type:'string'}

create_model_response = client.create_model(
    model_display_name,
    dataset=dataset,
    optimization_objective=model_optimization_objective,
    train_budget_milli_node_hours=model_train_hours * 1000,
)

print('Dataset import operation: {}'.format(create_model_response.operation))
# Wait until model training is done.
model = create_model_response.result()
model_name = model.name
model

In [0]:
metrics= [x for x in client.list_model_evaluations(model=model)][-1]
metrics.regression_evaluation_metrics

In [0]:
feat_list = [(x.feature_importance, x.column_display_name) for x in model.tables_model_metadata.tables_model_column_info]
feat_list.sort(reverse=True)
feat_list[:15]

4

In [118]:
dataset_display_name = 'stockout_data' #@param {type: 'string'}

dataset = client.create_dataset(dataset_display_name)
dataset

name: "projects/269327017718/locations/us-central1/datasets/TBL2877742987285102592"
display_name: "stockout_data"
create_time {
  seconds: 1573963134
  nanos: 747219000
}
etag: "AB3BwFrSafMX89tsBGi8hX-D5K8-EvAzBgPzyZjRIY57zzCKVtvZRVuulyDAbrh5AwzF"
tables_dataset_metadata {
  stats_update_time {
  }
}

In [123]:
import_data_operation = client.import_data(
    dataset=dataset,
    bigquery_input_uri=bq_input_uri,
)
print('Dataset import operation: {}'.format(import_data_operation))

NameError: ignored

In [0]:
import_data_operation.result()
dataset = client.get_dataset(dataset_name=dataset.name)
dataset

In [0]:
list_table_specs_response = client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]

# List column specs
list_column_specs_response = client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print Features and data_type:

features = [(key, data_types.TypeCode.Name(value.data_type.type_code)) for key, value in column_specs.items()]
print('Feature list:\n')
for feature in features:
    print(feature[0],':', feature[1])

In [0]:
type_counts = {}
for column_spec in column_specs.values():
  type_name = data_types.TypeCode.Name(column_spec.data_type.type_code)
  type_counts[type_name] = type_counts.get(type_name, 0) + 1
    
plt.pie(x=type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In [0]:
for column in column_specs:
    print(column, '-', column_specs[column].data_type)

In [0]:
categorical_column_names = ['Item_Number',
                        'Category',
                        'Vendor_Number',
                        'Store_Number',
                        'Zip_Code',
                        'County_Number']
is_nullable = [False, 
              False,
              False,
              False,
              True,
              True]

for i in range(len(categorical_column_names)):
    column_name = categorical_column_names[i]
    nullable = is_nullable[i]
    client.update_column_spec(
        dataset=dataset,
        column_spec_display_name=column_name,
        type_code='CATEGORY',
        nullable=nullable,
    )

In [0]:
target_column_name = 'Stockout' #@param {type: 'string'}
update_dataset_response = client.set_target_column(
    dataset=dataset,
    column_spec_display_name=target_column_name,
)
update_dataset_response

In [0]:
model_display_name = 'stockout_model' #@param {type:'string'}

create_model_response = client.create_model(
    model_display_name,
    dataset=dataset,
    train_budget_milli_node_hours=1000,
    optimization_objective='MAXIMIZE_AU_PRC',
)
print('Create model operation: {}'.format(create_model_response.operation))
# Wait until model training is done.
model = create_model_response.result()
model

In [0]:
batch_predict_bq_input_uri = 'bq://product-stockout.product_stockout.batch_prediction_inputs'
batch_predict_gcs_output_uri_prefix = 'gs://<BUCKET_NAME>'  #@param {type:'string'}

batch_predict_response = client.batch_predict(
    model=model, 
    biqquery_input_uris=batch_predict_bq_input_uri,
    gcs_output_uri_prefix=batch_predict_gcs_output_uri_prefix,
)
print('Batch prediction operation: {}'.format(batch_predict_response.operation))
# Wait until batch prediction is done.
batch_predict_result = batch_predict_response.result()
batch_predict_response.metadata

In [0]:
gcs_output_directory = batch_predict_response.metadata.batch_predict_details.output_info.gcs_output_directory
result_file = gcs_output_directory + 'tables_1.csv'
print('Batch prediction results are stored as: {}'.format(result_file))

5

In [0]:
! pip3 install --upgrade --quiet google-cloud-automl google-cloud-bigquery

In [0]:
from google.cloud import automl_v1beta1

In [0]:
PROJECT_ID = ""
# The region to use for compute resources (AutoML isn't supported in some regions).
LOCATION = "us-central1"
# A name for the AutoML tables Dataset to create.
DATASET_DISPLAY_NAME = ""
# The BigQuery dataset to import data from (doesn't need to exist).
INPUT_BQ_DATASET = ""
# The BigQuery table to import data from (doesn't need to exist).
INPUT_BQ_TABLE = ""
# A name for the AutoML tables model to create.
MODEL_DISPLAY_NAME = ""
# The number of hours to train the model.
MODEL_TRAIN_HOURS = 0

assert all([
    PROJECT_ID,
    LOCATION,
    DATASET_DISPLAY_NAME,
    INPUT_BQ_DATASET,
    INPUT_BQ_TABLE,
    MODEL_DISPLAY_NAME,
    MODEL_TRAIN_HOURS,
])

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from google.cloud import automl_v1beta1
from google.cloud import bigquery
from google.cloud import exceptions
import seaborn as sns

%matplotlib inline


tables_client = automl_v1beta1.TablesClient(project=PROJECT_ID, region=LOCATION)
bq_client = bigquery.Client()

In [0]:
query = """
   WITH
    songs AS (
      SELECT CONCAT(track_name, " by ", artist_name) AS song,
        MAX(tags) as tags
      FROM `listenbrainz.listenbrainz.listen`
      GROUP BY song
      HAVING tags != ""
      ORDER BY COUNT(*) DESC
      LIMIT 10000
    ),
    user_songs AS (
      SELECT user_name AS user, ANY_VALUE(artist_name) AS artist,
        CONCAT(track_name, " by ", artist_name) AS song,
        SPLIT(ANY_VALUE(songs.tags), ",") AS tags,
        COUNT(*) AS user_song_listens
      FROM `listenbrainz.listenbrainz.listen`
      JOIN songs ON songs.song = CONCAT(track_name, " by ", artist_name)
      GROUP BY user_name, song
    ),
    user_tags AS (
      SELECT user, tag, COUNT(*) AS COUNT
      FROM user_songs,
      UNNEST(tags) tag
      WHERE tag != ""
      GROUP BY user, tag
    ),
    top_tags AS (
      SELECT tag
      FROM user_tags
      GROUP BY tag
      ORDER BY SUM(count) DESC
      LIMIT 20
    ),
    tag_table AS (
      SELECT user, b.tag
      FROM user_tags a, top_tags b
      GROUP BY user, b.tag
    ),
    user_tag_features AS (
      SELECT user,
        ARRAY_AGG(IFNULL(count, 0) ORDER BY tag) as user_tags,
        SUM(count) as tag_count
      FROM tag_table
      LEFT JOIN user_tags USING (user, tag)
      GROUP BY user
    ), user_features AS (
      SELECT user, MAX(user_song_listens) AS user_max_listen,
        ANY_VALUE(user_tags)[OFFSET(0)]/ANY_VALUE(tag_count) as user_tags0,
        ANY_VALUE(user_tags)[OFFSET(1)]/ANY_VALUE(tag_count) as user_tags1,
        ANY_VALUE(user_tags)[OFFSET(2)]/ANY_VALUE(tag_count) as user_tags2,
        ANY_VALUE(user_tags)[OFFSET(3)]/ANY_VALUE(tag_count) as user_tags3,
        ANY_VALUE(user_tags)[OFFSET(4)]/ANY_VALUE(tag_count) as user_tags4,
        ANY_VALUE(user_tags)[OFFSET(5)]/ANY_VALUE(tag_count) as user_tags5,
        ANY_VALUE(user_tags)[OFFSET(6)]/ANY_VALUE(tag_count) as user_tags6,
        ANY_VALUE(user_tags)[OFFSET(7)]/ANY_VALUE(tag_count) as user_tags7,
        ANY_VALUE(user_tags)[OFFSET(8)]/ANY_VALUE(tag_count) as user_tags8,
        ANY_VALUE(user_tags)[OFFSET(9)]/ANY_VALUE(tag_count) as user_tags9,
        ANY_VALUE(user_tags)[OFFSET(10)]/ANY_VALUE(tag_count) as user_tags10,
        ANY_VALUE(user_tags)[OFFSET(11)]/ANY_VALUE(tag_count) as user_tags11,
        ANY_VALUE(user_tags)[OFFSET(12)]/ANY_VALUE(tag_count) as user_tags12,
        ANY_VALUE(user_tags)[OFFSET(13)]/ANY_VALUE(tag_count) as user_tags13,
        ANY_VALUE(user_tags)[OFFSET(14)]/ANY_VALUE(tag_count) as user_tags14,
        ANY_VALUE(user_tags)[OFFSET(15)]/ANY_VALUE(tag_count) as user_tags15,
        ANY_VALUE(user_tags)[OFFSET(16)]/ANY_VALUE(tag_count) as user_tags16,
        ANY_VALUE(user_tags)[OFFSET(17)]/ANY_VALUE(tag_count) as user_tags17,
        ANY_VALUE(user_tags)[OFFSET(18)]/ANY_VALUE(tag_count) as user_tags18,
        ANY_VALUE(user_tags)[OFFSET(19)]/ANY_VALUE(tag_count) as user_tags19
      FROM user_songs
      LEFT JOIN user_tag_features USING (user)
      GROUP BY user
      HAVING COUNT(*) < 5000 AND user_max_listen > 2
    ),
    item_features AS (
      SELECT CONCAT(track_name, " by ", artist_name) AS song,
        COUNT(DISTINCT(release_name)) AS albums
      FROM `listenbrainz.listenbrainz.listen`
      WHERE track_name != ""
      GROUP BY song
    )
  SELECT user, song, artist, tags, albums,
    user_tags0,
    user_tags1,
    user_tags2,
    user_tags3,
    user_tags4,
    user_tags5,
    user_tags6,
    user_tags7,
    user_tags8,
    user_tags9,
    user_tags10,
    user_tags11,
    user_tags12,
    user_tags13,
    user_tags14,
    user_tags15,
    user_tags16,
    user_tags17,
    user_tags18,
    user_tags19,
    IF(user_song_listens > 2, 
       SQRT(user_song_listens/user_max_listen), 
       .5/user_song_listens) AS weight,
    IF(user_song_listens > 2, 1, 0) as label
  FROM user_songs
  JOIN user_features USING(user)
  JOIN item_features USING(song)
"""

In [0]:
def create_table_from_query(query, table):
    """Creates a new table using the results from the given query.
    
    Args:
        query: a query string.
        table: a name to give the new table.
    """
    job_config = bigquery.QueryJobConfig()
    bq_dataset = bigquery.Dataset("{0}.{1}".format(PROJECT_ID, INPUT_BQ_DATASET))
    bq_dataset.location = "US"

    try:
        bq_dataset = bq_client.create_dataset(bq_dataset)
    except exceptions.Conflict:
        pass

    table_ref = bq_client.dataset(INPUT_BQ_DATASET).table(table)
    job_config.destination = table_ref

    query_job = bq_client.query(query,
                             location=bq_dataset.location,
                             job_config=job_config)

    query_job.result()
    print('Query results loaded to table {}'.format(table_ref.path))

In [0]:
create_table_from_query(query, INPUT_BQ_TABLE)

In [0]:
dataset = tables_client.create_dataset(
    dataset_display_name=DATASET_DISPLAY_NAME)

dataset_bq_input_uri = 'bq://{0}.{1}.{2}'.format(
    PROJECT_ID, INPUT_BQ_DATASET, INPUT_BQ_TABLE)
import_data_response = tables_client.import_data(
    dataset=dataset, bigquery_input_uri=dataset_bq_input_uri)
import_data_result = import_data_response.result()
import_data_result

In [0]:
    dataset_display_name=DATASET_DISPLAY_NAME)
column_specs = {s.display_name: s for s in list_column_specs_response}

def print_column_specs(column_specs):
    """Parses the given specs and prints each column and column type."""
    data_types = automl_v1beta1.proto.data_types_pb2
    return [(x, data_types.TypeCode.Name(
        column_specs[x].data_type.type_code)) for x in column_specs.keys()]

print_column_specs(column_specs)

In [0]:
for col in ["song", "artist"]:
    tables_client.update_column_spec(dataset_display_name=DATASET_DISPLAY_NAME,
                                     column_spec_display_name=col,
                                     type_code="CATEGORY")

list_column_specs_response = tables_client.list_column_specs(
    dataset_display_name=DATASET_DISPLAY_NAME)
column_specs = {s.display_name: s for s in list_column_specs_response}
print_column_specs(column_specs)

In [0]:
tables_client.set_target_column(dataset_display_name=DATASET_DISPLAY_NAME,
                                column_spec_display_name="label")

tables_client.set_weight_column(dataset_display_name=DATASET_DISPLAY_NAME,
                                column_spec_display_name="weight")

In [0]:
tables_client.create_model(
    model_display_name=MODEL_DISPLAY_NAME,
    dataset_display_name=DATASET_DISPLAY_NAME,
    train_budget_milli_node_hours= MODEL_TRAIN_HOURS * 1000).result()

In [0]:
users = ["rob", "fiveofoh", "Aerion"]
training_table = "{}.{}.{}".format(PROJECT_ID, INPUT_BQ_DATASET, INPUT_BQ_TABLE)
query = """
    WITH user as (
      SELECT user, 
        user_tags0, user_tags1, user_tags2, user_tags3, user_tags4,
        user_tags5, user_tags6, user_tags7, user_tags8, user_tags9,
        user_tags10,user_tags11, user_tags12, user_tags13, user_tags14,
        user_tags15, user_tags16, user_tags17, user_tags18, user_tags19, label
      FROM `{0}`
      WHERE user in ({1})
    )
    SELECT ANY_VALUE(a).*, song, ANY_VALUE(artist) as artist,
      ANY_VALUE(tags) as tags, ANY_VALUE(albums) as albums
    FROM `{0}`, user a
    GROUP BY song
""".format(training_table, ",".join(["\"{}\"".format(x) for x in users]))

In [0]:
eval_table = "{}_example".format(INPUT_BQ_TABLE)
create_table_from_query(query, eval_table)

In [0]:
preds_bq_input_uri = "bq://{}.{}.{}".format(PROJECT_ID, INPUT_BQ_DATASET, eval_table)
preds_bq_output_uri = "bq://{}".format(PROJECT_ID)
response = tables_client.batch_predict(model_display_name=MODEL_DISPLAY_NAME,
                                       bigquery_input_uri=preds_bq_input_uri,
                                       bigquery_output_uri=preds_bq_output_uri)
response.result()
output_uri = response.metadata.batch_predict_details.output_info.bigquery_output_dataset

In [0]:
n = 10
query = """
    SELECT user, song, tables.score as score, a.label as pred_label,
      b.label as true_label
    FROM `{}.predictions` a, UNNEST(predicted_label)
    LEFT JOIN `{}` b USING(user, song)
    WHERE user = "{}" AND CAST(tables.value AS INT64) = 1
    ORDER BY score DESC
    LIMIT {}
""".format(output_uri[5:].replace(":", "."), training_table, users[0], n)
query_job = bq_client.query(query)

print("Top {} song recommended for {}:".format(n, users[0]))
for idx, row in enumerate(query_job):
    print("{}.".format(idx + 1), row["song"])

In [0]:
query = """
    WITH 
      top_k AS (
        SELECT user, song, label,
          ROW_NUMBER() OVER (PARTITION BY user ORDER BY label + weight DESC) as user_rank
        FROM `{0}`
      )
    SELECT user, song, tables.score as score, b.label,
      ROW_NUMBER() OVER (ORDER BY tables.score DESC) as rank, user_rank
    FROM `{1}.predictions` a, UNNEST(predicted_label)
    LEFT JOIN top_k b USING(user, song)
    WHERE CAST(tables.value AS INT64) = 1
    ORDER BY score DESC
""".format(training_table, output_uri[5:].replace(":", "."))

df = bq_client.query(query).result().to_dataframe()
df.head()

In [0]:
precision_at_k = {}
recall_at_k = {}

for user in users:
    precision_at_k[user] = []
    recall_at_k[user] = []
    for k in range(1, 1000):
        precision = df["label"][:k].sum() / k
        recall = df["label"][:k].sum() / df["label"].sum()
        precision_at_k[user].append(precision)
        recall_at_k[user].append(recall)

# plot the precision-recall curve
ax = sns.lineplot(recall_at_k[users[0]], precision_at_k[users[0]])
ax.set_title("precision-recall curve for varying k")
ax.set_xlabel("recall@k")
ax.set_ylabel("precision@k")

In [0]:
def calculate_ap(precision):
    ap = [precision[0]]
    for p in precision[1:]:
        ap.append(ap[-1] + p)
    ap = [x / (n + 1) for x, n in zip(ap, range(len(ap)))]
    return ap

ap_at_k = {user: calculate_ap(pk)
           for user, pk in precision_at_k.items()}

num_k = 500
map_at_k = [sum([ap_at_k[user][k] for user in users]) / len(users)
            for k in range(num_k)]
print("MAP@50: {}".format(map_at_k[49]))

# plot average precision
ax = sns.lineplot(range(num_k), map_at_k)
ax.set_title("MAP@k for varying k")
ax.set_xlabel("k")
ax.set_ylabel("MAP")